# CRFを用いた固有表現認識

このノートブックでは、[sklearn-crfsuite](https://sklearn-crfsuite.readthedocs.io/en/latest/)を用いて、固有表現認識のモデルを構築します。データセットとしては　CoNLLを使用します。

## 準備

### パッケージのインストール

In [1]:
!pip install sklearn-crfsuite==0.3.6 seqeval==1.2.2 eli5==0.11.0 scikit-learn==0.23.2

### インポート

In [2]:
import eli5
import scipy
from seqeval.metrics import classification_report, f1_score
from sklearn_crfsuite import CRF
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

### データの読み込み

まずはCoNLLのデータセットをアップロードします。ノートブックと同じ階層にDataフォルダがあり、その下に`conll2003/en`フォルダがあるので、学習・検証・テスト用データセットをアップロードしましょう。Colabでない場合は、データセットを読み込むときに正しいパスを指定します。

In [3]:
from google.colab import files
uploaded = files.upload()

Saving test.txt to test (1).txt
Saving train.txt to train (1).txt
Saving valid.txt to valid (1).txt


In [4]:
!head train.txt

-DOCSTART- -X- -X- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O


次に、データセットを読み込みます。

In [5]:
def load_conll(file_path):
    sents = []
    sent = []
    with open(file_path, encoding='utf-8') as f:
      for line in f:
          line = line.strip()
          if line.startswith('-DOCSTART'):
            continue
          if line:
              word, pos, _, tag = line.split()
              sent.append((word, pos, tag))
          else:
              if len(sent) == 0:
                continue
              sents.append(sent)
              sent = []
    return sents

In [6]:
train_sents = load_conll('train.txt')
valid_sents = load_conll('valid.txt')
test_sents = load_conll('test.txt')

In [7]:
train_sents[0]

[('EU', 'NNP', 'B-ORG'),
 ('rejects', 'VBZ', 'O'),
 ('German', 'JJ', 'B-MISC'),
 ('call', 'NN', 'O'),
 ('to', 'TO', 'O'),
 ('boycott', 'VB', 'O'),
 ('British', 'JJ', 'B-MISC'),
 ('lamb', 'NN', 'O'),
 ('.', '.', 'O')]

In [8]:
print(len(train_sents))
print(len(valid_sents))
print(len(test_sents))

14041
3250
3453


## 前処理

データを読み込み終えたので、特徴を定義します。今回は前後2単語に関して、以下の特徴を使います。

- 小文字化した単語
- 大文字だけからなるか
- 単語の先頭の文字は大文字か
- 数字か
- 品詞

In [9]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.istitle()': word1.istitle(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True

    if i > 1:
        word2 = sent[i-2][0]
        postag2 = sent[i-2][1]
        features.update({
            '-2:word.lower()': word2.lower(),
            '-2:word.isupper()': word2.isupper(),
            '-2:word.istitle()': word2.istitle(),
            '-2:postag': postag2,
        })
    else:
        features['-2:BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.istitle()': word1.istitle(),
            '+1:postag': postag1,
        })
    else:
        features['EOS'] = True

    if i < len(sent) - 2:
        word2 = sent[i+2][0]
        postag2 = sent[i+2][1]
        features.update({
            '+2:word.lower()': word2.lower(),
            '+2:word.isupper()': word2.isupper(),
            '+2:word.istitle()': word2.istitle(),
            '+2:postag': postag2,
        })
    else:
        features['+2:EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [10]:
sent2features(train_sents[0])[0]

{'+1:postag': 'VBZ',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'rejects',
 '+2:postag': 'JJ',
 '+2:word.istitle()': True,
 '+2:word.isupper()': False,
 '+2:word.lower()': 'german',
 '-2:BOS': True,
 'BOS': True,
 'bias': 1.0,
 'postag': 'NNP',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': True,
 'word.lower()': 'eu'}

In [11]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_valid = [sent2features(s) for s in valid_sents]
y_valid = [sent2labels(s) for s in valid_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

## モデルの学習

In [12]:
%%time
model = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=False
)
model.fit(X_train + X_valid, y_train + y_valid)

CPU times: user 46.8 s, sys: 240 ms, total: 47 s
Wall time: 46.9 s


## モデルの評価

In [13]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

         LOC     0.8385    0.7842    0.8104      1668
        MISC     0.7602    0.6638    0.7087       702
         ORG     0.7256    0.7038    0.7145      1661
         PER     0.8041    0.8683    0.8350      1617

   micro avg     0.7861    0.7697    0.7778      5648
   macro avg     0.7821    0.7550    0.7672      5648
weighted avg     0.7857    0.7697    0.7766      5648



## 重みの検査

[eli5](https://eli5.readthedocs.io/en/latest/index.html)の`show_weights`を使って、状態と繊維の特徴に関する重みを見てみましょう。


In [14]:
eli5.show_weights(model, top=30)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


遷移の行列を見ると、モデルはB-XのあとにI-Xが来ることを学習していることがわかります。また、ある種の遷移はほとんど起こらないことも学習しているようです。たとえば、B-LOCのあとにI-ORGが来ることはほとんどないといったことです。

## ハイパーパラメータの最適化

性能を向上させるに、ハイパーパラメータの最適化をしてみましょう。ここでは、ランダムサーチと3分割交差検定を使用して、正則化パラメータの探索をしてみます。

In [15]:
%%time
model = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(f1_score)

rs = RandomizedSearchCV(
    model,
    params_space,
    cv=3,
    verbose=1,
    n_jobs=-1,
    n_iter=30,
    scoring=f1_scorer
)
rs.fit(X_train + X_valid, y_train + y_valid)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 37.9min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 73.3min finished


CPU times: user 1h 3min 9s, sys: 13.1 s, total: 1h 3min 22s
Wall time: 1h 14min 1s


In [16]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.028100798861946483, 'c2': 0.002451948365671623}
best CV score: 0.8532850512553268
model size: 1.61M


テストデータを使って、最適なパラメータのモデルを評価してみましょう。

In [17]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

         LOC     0.8767    0.8483    0.8623      1668
        MISC     0.7868    0.7151    0.7493       702
         ORG     0.7815    0.7363    0.7582      1661
         PER     0.8395    0.8862    0.8622      1617

   micro avg     0.8278    0.8097    0.8187      5648
   macro avg     0.8211    0.7965    0.8080      5648
weighted avg     0.8269    0.8097    0.8176      5648

